# Biostat 257 Homework 3

**Due May 15 @ 11:59PM**

Ian Frankenburg

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In [2]:
using CodecBzip2, JuliaDB, CSV, IterableTables, SweepOperator, LinearAlgebra, 
    MatrixDepot, UnicodePlots, SparseArrays, IterativeSolvers, LinearMaps, 
    Statistics, Arpack, LinearAlgebra

include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


## Q1. (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

I want an element $p_{ij}$ of the transition matrix $\mathbf P$, which represents the probability of moving from page $i$ to page $j$. I have two cases to consider: case one will be when page $i$ contains external links, i.e., $r_i>0$. Case two will be when page $i$ contains no links, i.e., $r_i=0$. Therefore, $p_{ij}$ is given as
$$
p_{ij}=
\begin{cases}
p\frac{1}{r_i}+(1-p)\frac{1}{n}&\text{if }r_i>0\\
\frac{1}{n}&\text{if }r_i=0
\end{cases}
$$

Now to write this as a sparse matrix plus a rank-one matrix, I define diagonal matrix $\mathbf D$ with elements  
$$
d_{ii}:=
\begin{cases}
\frac{p}{r_i}&\text{ if }r_i\neq 0\\
0&\text{ otherwise}
\end{cases}
$$
Then I define the vector $\mathbf z$ to form the needed rank-one matrix. Let component $j$ of $\mathbf z$ be defined as
$$
z_j :=\begin{cases}
\frac{(1-p)}{n}&\text{ if } r_i\neq 0\\
\frac{1}{n}&\text{ otherwise}
\end{cases}
$$
Then
$$
\mathbf P=\mathbf{DA}+\mathbf{z}\mathbf{1}^{\top}
$$

## Q2. Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3. (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [3]:
using MatrixDepot
md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [4]:
# connectivity matrix
MatrixDepot.addmetadata!(md.data)
A = md.A

916428×916428 SparseMatrixCSC{Bool,Int64} with 5105039 stored entries:
  [11343 ,      1]  =  1
  [11928 ,      1]  =  1
  [15902 ,      1]  =  1
  [29547 ,      1]  =  1
  [30282 ,      1]  =  1
  [31301 ,      1]  =  1
  [38717 ,      1]  =  1
  [43930 ,      1]  =  1
  [46275 ,      1]  =  1
  [48193 ,      1]  =  1
  [50823 ,      1]  =  1
  [56911 ,      1]  =  1
  ⋮
  [608625, 916427]  =  1
  [618730, 916427]  =  1
  [622998, 916427]  =  1
  [673046, 916427]  =  1
  [716616, 916427]  =  1
  [720325, 916427]  =  1
  [772226, 916427]  =  1
  [785097, 916427]  =  1
  [788476, 916427]  =  1
  [822938, 916427]  =  1
  [833616, 916427]  =  1
  [417498, 916428]  =  1
  [843845, 916428]  =  1

Compute summary statistics:
* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  
* number of web pages
* number of edges (web links). 
* number of dangling nodes (pages with no out links)
* histogram of in-degrees  **(TODO)**
* list the top 20 pages with the largest in-degrees?  **(TODO)**
* histogram of out-degrees **(TODO)**
* which the top 20 pages with the largest out-degrees?**(TODO)**
* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`.**(TODO)**

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

## Q3 Solution
* If `A` is converted to a `Matrix{Float64}`, it will take $916428\times916428\times8$ byes. As a sparse matrix, `A` takes $5105039\times 8$ bytes.

In [5]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise
@benchmark A

#Number of pages
num_pages = size(A)[1]
@show num_pages

#Number of edges (web links)
num_edges = sum(A)
@show num_edges

#Number of dangling nodes (pages with no out links)
num_dangling = 0
AT = sparse(transpose(A))
for page in 1:size(A)[1]
    if sum(AT[:, page]) == 0
        num_dangling = num_dangling + 1
    end
end

@show num_dangling

num_pages = 916428
num_edges = 5105039
num_dangling = 176974


176974

In [43]:
in_degrees = zeros(1, size(A)[2])

for page in 1:size(A)[2]
    in_degrees[1, page] = sum(A[:, page])
end

out_degrees = zeros(1, size(A)[1])

for page in 1:size(A)[1]
    out_degrees[1, page] = sum(AT[:, page])
end

top_20_in = sortperm(vec(in_degrees), rev = true)[1:20]
top_20_out = sortperm(vec(out_degrees), rev = true)[1]
@show top_20_in
@show top_20_out

max_in_page = (sortperm(vec(in_degrees), rev = true))[1:20] = [537040, 597622, 504141, 751385, 32164, 885606, 163076, 819224, 605857, 828964, 551830, 41910, 558792, 459075, 407611, 213433, 765335, 384667, 173977, 687326]
max_out_page = (sortperm(vec(out_degrees), rev = true))[1] = 506743


506743

In [7]:
#Sparsity pattern in A
# TO DO
using UnicodePlots
# spy(A) 

## Q4. (5 pts) Dense linear algebra? (TODO)

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

## Q5. Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide is methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [96]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A           :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep       :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    z           :: Vector{T}
    d           :: Vector{T}
    temp_store  :: Vector{T}
    temp_store2 :: Vector{T}
    ones        :: Vector{T}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    # TODO: initialize and pre-allocate arrays
    n = size(A, 1);
    r = A * ones(n)
    z = zeros(n);
    d = similar(z)
    for i in 1:n
        if r[i] == 0
            d[i], z[i] = 0, 1 / n
        else
            d[i], z[i] = telep / r[i], (1 - telep) / n
        end
    end
    temp_store = similar(z);
    temp_store2 = similar(z);
    one = ones(n);
    PageRankImPt(A, telep, z, d, one, temp_store, temp_store2)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)

# TODO: implement this function for evaluating M[i, j]
function Base.getindex(M::PageRankImPt, i, j)
    if i == j
        out = 1 - M.A[j, i] * M.d[i] - M.z[j]
    else
        out = - M.A[j, i] * M.d[i] - M.z[j]
    end
    return out
end

# overwrite `out` by `(I - P') * v` = `Mv`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    # get z'v
    ztv = dot(M.z,v)
    # get D'v
    M.temp_store .= M.d .* v
    # get A'D'v
    mul!(M.temp_store2, transpose(M.A), M.temp_store)
    # return v - A'D'v - 1z'v
    #out .= v .- M.temp_store2 .- M.temp_store
    out .= v .- M.temp_store2 .- ztv
    return out
end

# overwrite `out` by `(I - P) * v`=`M'v
# putting in the P from before: v-DAv-z1'v
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    # get Av
    mul!(M.temp_store, M.A, v)
    # get DAv
    M.temp_store .*= M.d
    # get 1'v
    vsum = sum(v)
    # get z1'v
    M.temp_store2 .= M.z .* vsum
    # get DAv-z1'v
    M.temp_store .+= M.temp_store2
    out .= v .- M.temp_store
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M}^T \mathbf{x} = \mathbf{x}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv`. **Do not put this file in your Git**. You can add a line `pgrksol.csv` to your `.gitignore` file.

In [102]:
using DelimitedFiles

isfile("pgrksol.csv") || download("https://raw.githubusercontent.com/ucla-biostat-257-2020spring/ucla-biostat-257-2020spring.github.io/master/hw/hw3/pgrksol.csv", "./pgrksol.csv")
xsol = vec(readdlm("pgrksol.csv"))

916428-element Array{Float64,1}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [103]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [104]:
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate litte memory.

In [105]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     22.375 ms (0.00% GC)
  median time:      22.907 ms (0.00% GC)
  mean time:        23.808 ms (0.00% GC)
  maximum time:     31.181 ms (0.00% GC)
  --------------
  samples:          205
  evals/sample:     1

In [106]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     30.668 ms (0.00% GC)
  median time:      33.202 ms (0.00% GC)
  mean time:        34.507 ms (0.00% GC)
  maximum time:     42.400 ms (0.00% GC)
  --------------
  samples:          142
  evals/sample:     1

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [107]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

19.84

**Hint**: My median run times are 30-40 ms and memory allocations are 0-16 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [32]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
n = size(A, 1)
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(([3.3783428221888516e-5, 2.0710155392484304e-6, 3.663065985237116e-6, 7.527510785602617e-7, 8.633285997161303e-7, 1.7694182527377874e-6, 2.4312303829121483e-7, 6.368417180747212e-7, 4.7449737037499775e-7, 2.6895486110965613e-7  …  3.1679469739776804e-6, 9.726880410231163e-7, 6.588614478542447e-7, 7.73701177471464e-7, 2.4312303829121483e-7, 1.6219204214246199e-6, 3.9121300606314574e-7, 2.4312303829121483e-7, 7.296033831324933e-6, 6.330939996678431e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821167768423286e-13,).
), 3.969964716, 1010825081, 0.090475435, Base.GC_Diff(1010825081, 137, 0, 123430, 6, 0, 90475435, 2, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [33]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [34]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 8 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [35]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

((Complex{Float64}[-1.4453413852555562e-14 + 0.0im], Array{Complex{Float64},1}[[0.0056358269538073135 + 0.0im, 0.00034549143807828807 + 0.0im, 0.0006110808494125191 + 0.0im, 0.0001255756162603543 + 0.0im, 0.00014402240532803497 + 0.0im, 0.00029517830503985064 + 0.0im, 4.055832828678035e-5 + 0.0im, 0.00010623935784852223 + 0.0im, 7.915671116195587e-5 + 0.0im, 4.486765066725073e-5 + 0.0im  …  0.0005284839899797014 + 0.0im, 0.00016226599147988767 + 0.0im, 0.00010991273837609865 + 0.0im, 0.0001290705585575189 + 0.0im, 4.055832828678035e-5 + 0.0im, 0.0002705723874320433 + 0.0im, 6.526302748333534e-5 + 0.0im, 4.0558328286780356e-5 + 0.0im, 0.001217140660132839 + 0.0im, 0.00010561415510699434 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.577598665649965e-14,).
), 7.238383239, 1584030677, 0.185964554, Base.GC_Diff(1584030677, 213, 0, 139759, 50, 0, 185964554, 6, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [36]:
@assert abs(Real(vals[1])) < 1e-8

In [37]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [38]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 11 seconds.

## Q6. (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

In [40]:
sortperm(vec(x_gmres), rev = true)[1:20]

20-element Array{Int64,1}:
 597622
  41910
 163076
 537040
 384667
 504141
 486981
 605857
  32164
 558792
 551830
 765335
 751385
 425771
 908352
 173977
   7315
 213433
 885606
 819224

In [41]:
sortperm(vec(x_arnoldi), rev = true)[1:20]

20-element Array{Int64,1}:
 597622
  41910
 163076
 537040
 384667
 504141
 486981
 605857
  32164
 558792
 551830
 765335
 751385
 425771
 908352
 173977
   7315
 213433
 885606
 819224

## Q7. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.